# Shot noise sim from ALFALFA HIMF

This is an attempt to make some sort of simulation of HI galaxies from the ALFALFA HI mass function. I think this function is the differential of the number density of galaxies of mass M.

In [43]:
%matplotlib inline

import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats

Take Schechter function parameters from Martin et al. 2010. Explore HI mass range from 6 to 11 solar masses.

In [44]:
# bunch of constants/parameters
h = 6.62607e-34
c = 2.99792e8
A = 2.869e-15
m = 1.67372e-27
k = 1.38064e-23
v = 1.42041e9
z = 0.08
H = 2.26719e-18
rhoc = 2.49732453e41 # in kg/Mpc^3
#rhoc = 2.2444834e58

# conversion from M_HI to T_b
C = 3 * h * c**3 * A * (1+z)**2 / (31*np.pi * m * k * v**2 * H)
# volume of box in metres (convert from Mpc)
V = (139 * 1.e6 * 3.08568025e16)**3
Nv = 139
# solar mass
Ms = 1.989e30

In [45]:
phi, mstar, alpha = 8.6e-3, 10.**9.79, -1.30

x = np.logspace(np.log10(6), np.log10(11), 240000)

We take a conventional definition of the Schechter function. Note that it's often given as a differential, so integration is required to return the proper form. 

In [46]:
#mf = phi * (x / mstar)**(1 + alpha) * np.exp(-x / mstar) / (-1 - alpha)
mf =  phi * (10**x / mstar)**(alpha + 1) *\
         np.exp(-10**x / mstar)

In [105]:
# PLOT the HIMF

#plt.plot(x, mf)
#plt.xlim([6.4, 11.])
#plt.ylim([1e-6, 1.e0])
#plt.xlabel('log$(M_{{HI}}/M_\odot)$')
#plt.ylabel('$\Phi(M_{{HI}})$ [$Mpc^{-3}$]')
#plt.yscale('log')
#plt.show()

Now we choose mass bins over which we'll integrate the function to return the mass density of galaxies for each bin. The function integrated over the entire domain is the HI mass density; divided by the critical density, this is $\Omega_\text{HI}$.

In [91]:
bins = np.logspace((6), (11), 50)
nbin = np.ones_like(bins[:-1])
cent = np.ones_like(bins[:-1])
var = 10**x

for bb in range(len(bins)-1):
    cent[bb] *= (bins[bb] + bins[bb+1]) / 2   
    mask = (var>=bins[bb]) & (var<bins[bb+1])
    nbin[bb] *= (integrate.simps(mf[mask], var[mask])) / cent[bb]

rho = integrate.simps(mf, var)

[ 0.02658194  0.02476688  0.02308382  0.02150657  0.02004182  0.0186762
  0.01740328  0.01621412  0.01510873  0.01407389]
1.60526315789e+11 1.25556788839e+11 68663787.587


In [106]:
probs = np.append(nbin, 1-np.sum(nbin))
masses = np.append(cent, 0)

In [107]:
lbox = 139
box = np.random.choice(masses, size=(lbox,lbox,lbox), p=probs)

#plt.hist(box.flatten(), bins=bins)
#plt.xlabel('log$(M_{{HI}}/M_\odot)$')
#plt.ylabel('N')
#plt.show()

For reference, we'll now compute the brightness temperature of each voxel, given by

$ T_{\text{HI}}(x_i) = C \, M(x_i)(N_v/V)$,

where $M(x_i)$ is the HI mass in the $i^{\text{th}}$ voxel, $N_v$ is the number of voxels, and 

$C = \frac{3hc^3 A_{10}}{32\pi m_H k_B v^2_{21}}\frac{(1+z)^2}{H(z)}$,

where $h$ is Planck's constant, $A_{10}$ is the Einstein coefficient, and $m_H$ is the mass of the H atom.

In [108]:
h = 6.62607e-34
c = 2.99792e8
A = 2.869e-15
m = 1.67372e-27
k = 1.38064e-23
v = 1.42041e9
z = 0.08
H = 2.26719e-18

C = 3 * h * c**3 * A * (1+z)**2 / (31*np.pi * m * k * v**2 * H)
# volume of box in metres (convert from Mpc)
V = (139 * 1.e6 * 3.08568025e16)**3
Nv = 139
# solar mass
Ms = 1.989e30

# convert box of masses to kg 
massbox = (box)
massbox[box==0] = 0
# compute box of brightness temps
Tb = C * massbox * Ms * Nv**3 / V

Above we computed the brightness temperature of the HI in each voxel. Below we'll perform a sanity check and make a histogram of the temps.

In [103]:
tempbins = np.logspace(np.log10(1.e-6), np.log10(2.e-1), 20)
#plt.hist(Tb.flatten(), bins=tempbins)
#plt.ylabel('N')
#plt.xlabel('$T_{\mathrm{b}}\, $[K]')
#plt.xscale('log')
#plt.yscale('log')

In [109]:
T_b = np.mean(Tb)
T_b

8.1320728302309913e-05

This is a tad too high... to be continued.